In [ ]:
msrvtt10k_files = "/home/mrim/tools/irim/pytorch/danieljf24/VisualSearch/msrvtt10k_files/results_"
# concept_ext_model_best = "/msrvtt10k_model_best.pth.tar/"
concept_ext_model_best = "_concept_only/model_best_concept.pth/"
# concept_ext_model_best = "_cxd/model_best_concept_cxd.pth/"

import numpy as np
splits = ['train', 'test', 'val']
scores_cap, scores_vid = [], []
scores_cap.append(np.load(msrvtt10k_files + "train" + concept_ext_model_best + "pre_sig_cap_prob_train.npy"))
scores_vid.append(np.load(msrvtt10k_files + "train" + concept_ext_model_best + "pre_sig_vid_prob_train.npy"))
scores_cap.append(np.load(msrvtt10k_files + "test" + concept_ext_model_best + "pre_sig_cap_prob_test.npy"))
scores_vid.append(np.load(msrvtt10k_files + "test" + concept_ext_model_best + "pre_sig_vid_prob_test.npy"))
scores_cap.append(np.load(msrvtt10k_files + "val" + concept_ext_model_best + "pre_sig_cap_prob_val.npy"))
scores_vid.append(np.load(msrvtt10k_files + "val" + concept_ext_model_best + "pre_sig_vid_prob_val.npy"))
labels_cap, labels_vid = [], []
labels_cap.append(np.load(msrvtt10k_files + "train/msrvtt10k_model_best.pth.tar/train_sent_binary_labels.npy"))
labels_vid.append(np.load(msrvtt10k_files + "train/msrvtt10k_model_best.pth.tar/train_vid_soft_labels.npy"))
labels_cap.append(np.load(msrvtt10k_files + "test/msrvtt10k_model_best.pth.tar/test_sent_binary_labels.npy"))
labels_vid.append(np.load(msrvtt10k_files + "test/msrvtt10k_model_best.pth.tar/test_vid_soft_labels.npy"))
labels_cap.append(np.load(msrvtt10k_files + "val/msrvtt10k_model_best.pth.tar/val_sent_binary_labels.npy"))
labels_vid.append(np.load(msrvtt10k_files + "val/msrvtt10k_model_best.pth.tar/val_vid_soft_labels.npy"))
print(scores_cap[0].shape, scores_cap[1].shape, scores_cap[2].shape)
print(scores_vid[0].shape, scores_vid[1].shape, scores_vid[2].shape)
n_concepts = scores_cap[0].shape[1]

from itertools import islice
with open(msrvtt10k_files + "train/msrvtt10k_model_best.pth.tar/all_words.txt") as f:
    cc=[line.split() for line in islice(f, n_concepts)]
cname = [c[0] for c in cc]
ccount = [int(c[1]) for c in cc]

In [ ]:
np.set_printoptions(precision=3, threshold=None, edgeitems=None, linewidth=100, suppress=True, formatter=None,
                    sign=None, floatmode='fixed')

In [ ]:
soft_th = 0.07  # Soft labels threshold
hlabels_vid = [labels_vid_split > soft_th for labels_vid_split in labels_vid]

from sklearn.metrics import average_precision_score
import warnings

def get_data_one(scores, labels, j, c):
    with np.errstate(divide='ignore', invalid='ignore'):
        ap = average_precision_score(labels[j][:,c], scores[j][:,c], average=None)
    scores0 = scores[j][:,c][labels[j][:,c] == 0.0]
    scores1 = scores[j][:,c][labels[j][:,c] == 1.0]
    n0, n1 = scores0.shape[0], scores1.shape[0]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        mu0, mu1 = np.mean(scores0), np.mean(scores1)
    if n0 > 1 and n1 > 1:
        sigma0 = np.sqrt((np.square(scores0-mu0).sum())/(n0-1))
        sigma1 = np.sqrt((np.square(scores1-mu1).sum())/(n1-1))
        z = (mu1-mu0)/(sigma0+sigma1)
    else: z = np.nan
    sigma = np.sqrt((np.square(scores0-mu0).sum()+np.square(scores1-mu1).sum())/(n0+n1-1))
    a = (mu1-mu0)/(sigma*sigma)
    with np.errstate(divide='ignore', invalid='ignore'):
        b = np.log(np.float64(n1)/np.float64(n0))-0.5*(mu1*mu1-mu0*mu0)/(sigma*sigma)
        r = np.float64(n0)/np.float64(n1)
    return(scores0, scores1, ap, z, n0, n1, mu0, mu1, sigma, a, b, r)

def get_data(scores, labels, splits, cname):
    return([[get_data_one(scores, labels, j, c) for c in range(len(cname))] for j in range(len(splits))])

data_cap = get_data(scores_cap, labels_cap, splits, cname)
data_vid = get_data(scores_vid, hlabels_vid, splits, cname)
z_cap = np.array([[data_cap[j][c][3] for c in range(len(cname))] for j in range(len(splits))])
z_vid = np.array([[data_vid[j][c][3] for c in range(len(cname))] for j in range(len(splits))])
ap_cap = np.array([[data_cap[j][c][2] for c in range(len(cname))] for j in range(len(splits))])
ap_vid = np.array([[data_vid[j][c][2] for c in range(len(cname))] for j in range(len(splits))])
map_cap = np.nanmean(ap_cap, axis=1)
map_vid = np.nanmean(ap_vid, axis=1)
print("MAP   train    test    val")
print("cap   %.3f   %.3f   %.3f" % (map_cap[0], map_cap[1], map_cap[2]))
print("vid   %.3f   %.3f   %.3f" % (map_vid[0], map_vid[1], map_vid[2]))

In [ ]:
import seaborn as sns
sns.histplot([scores_cap[0][:,0], scores_vid[0][:,0]], bins=40, binrange=[-4,6], kde=True,
             element='step', stat='probability', fill=False, common_norm=False)

In [ ]:
sns.histplot([scores_cap[1][:,0], scores_vid[1][:,0]], bins=40, binrange=[-4,6], kde=True,
             element='step', stat='probability', fill=False, common_norm=False)

In [ ]:
sns.histplot([scores_cap[0][:,-1], scores_vid[0][:,-1]], bins=40, binrange=[-4,6], kde=True,
             element='step', stat='probability', fill=False, common_norm=False)

In [ ]:
sns.histplot([scores_cap[1][:,-1], scores_vid[1][:,-1]], bins=40, binrange=[-4,6], kde=True,
             element='step', stat='probability', fill=False, common_norm=False)

In [ ]:
sns.kdeplot([scores_cap[0][:,0], scores_vid[0][:,0]], clip=[-4,6], bw_adjust=2.0, common_norm=False)

In [ ]:
sns.kdeplot([scores_cap[1][:,0], scores_vid[1][:,0]], clip=[-4,6], bw_adjust=2.0, common_norm=False)

In [ ]:
sns.kdeplot([scores_cap[0][:,-1], scores_vid[0][:,-1]], clip=[-4,6], bw_adjust=2.0, common_norm=False)

In [ ]:
sns.kdeplot([scores_cap[1][:,-1], scores_vid[1][:,-1]], clip=[-4,6], bw_adjust=2.0, common_norm=False)

In [ ]:
print(labels_cap[0].shape)
print(labels_vid[0].shape)
import matplotlib.pyplot as plt
plt.hist(labels_vid[0].flatten(), bins=21, range=[-0.025,1.025], histtype='step')

In [ ]:
plt.hist(labels_vid[0].flatten(), bins=20, range=[0.025,1.025], histtype='step')

In [ ]:
plt.hist(labels_vid[1].flatten(), bins=21, range=[-0.025,1.025], histtype='step')

In [ ]:
plt.hist(labels_vid[1].flatten(), bins=20, range=[0.025,1.025], histtype='step')

In [ ]:
fig, axes = plt.subplots(figsize=(8,4), ncols=1, nrows=1)
axes.set_xlim(-4, 6)
data = []
for j in range(3): data.append(scores_cap[j][:,-1][labels_cap[j][:,-1] == 0.0])
for j in range(3): data.append(scores_cap[j][:,-1][labels_cap[j][:,-1] == 1.0])
sns.kdeplot(data, clip=[-4,6], bw_adjust=1.0, common_norm=False, ax = axes)
fig.show()

In [ ]:
fig, axes = plt.subplots(figsize=(8,4), ncols=1, nrows=1)
axes.set_xlim(-4, 6)
data = []
for j in range(3): data.append(scores_vid[j][:,-1][labels_vid[j][:,-1] == 0.0])
for j in range(3): data.append(scores_vid[j][:,-1][labels_vid[j][:,-1] == 1.0])
sns.kdeplot(data, clip=[-4,6], bw_adjust=1.0, common_norm=False, ax = axes)
fig.show()

In [ ]:
def plot_kde(data, cnum, bounds=[-4,10]):
    if not isinstance(cnum, list): cnum = [cnum]
    l = len(cnum)
    fig, axes = plt.subplots(l, 3, figsize=(12,2.5*l), squeeze=False)
    fig.tight_layout(h_pad = 3.0, w_pad = 0.5)
    xlabel_format = "n: %d, p: %d, r: %.2f, ap: %.4f, z: %.2f\n" \
    + "\u03BC0: %.2f, \u03BC1: %.2f, \u03C3: %.2f, a: %.2f, b: %.2f, c: %.2f"
    for i, c in enumerate(cnum):
        for j, s in enumerate(splits):
            scores0, scores1, ap, z, n0, n1, mu0, mu1, sigma, a, b, r = data[j][c]
            axes[i][j].set_xlim(bounds[0], bounds[1])
            if not i: axes[i][j].set_title(s)
            axes[i][j].set(xlabel = xlabel_format % (n0, n1, r, ap, z, mu0, mu1, sigma, a, b, -b/a))
            sns.kdeplot(scores0, color = 'r', clip=bounds, bw_adjust=1.0,
                        warn_singular=False, common_norm=False, legend=True, ax = axes[i][j], label = 'neg')
            sns.kdeplot(scores1, color = 'g', clip=bounds, bw_adjust=1.0,
                        warn_singular=False, common_norm=False, legend=True, ax = axes[i][j], label = 'pos')
            axes[i][j].legend()
            if j: axes[i][j].set(ylabel=None)
            else: axes[i][j].set(ylabel="%d %s" % (cnum[i]+1, cname[cnum[i]]))
            axes[i][j].xaxis.set_label_position('top') 
    fig.show()

In [ ]:
# rnum_cap = np.argsort((z_cap[1]+z_cap[2])/z_cap[0])
znum_cap = np.argsort(-z_cap, axis=1)
anum_cap = np.argsort(-ap_cap, axis=1)
plot_kde(data_cap, list(anum_cap[0][0:6]))

In [ ]:
plot_kde(data_cap, list(anum_cap[0][n_concepts//2-3:n_concepts//2+3]))

In [ ]:
plot_kde(data_cap, list(anum_cap[0][n_concepts-6:n_concepts]))

In [ ]:
znum_vid = np.argsort(-z_vid, axis=1)
anum_vid = np.argsort(-ap_vid, axis=1)
plot_kde(data_vid, list(anum_vid[0][0:6]))

In [ ]:
plot_kde(data_vid, list(anum_vid[0][n_concepts//2-3:n_concepts//2+3]))

In [ ]:
plot_kde(data_vid, list(anum_vid[0][n_concepts-6:n_concepts]))

In [ ]:
plot_kde(data_cap, [101, 387, 499])

In [ ]:
plot_kde(data_vid, [0, 101, 277])

In [ ]:
# plot_kde(data_cap, list(rnum_cap[0:20]))